# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark RDD - SOLUTION
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful

In [1]:
from pyspark import SparkContext, SparkConf
import numpy as np
import operator

In [2]:
conf=SparkConf().setAppName("Lab4-rdd").setMaster("local[*]")
sc = SparkContext(conf=conf)

Using PySpark and RDD's on the https://coding.csel.io machines is slow -- most of the code is executed in Python and this is much less efficient than the java-based code using the PySpark dataframes. Be patient and trying using `.cache()` to cache the output of joins. You may want to start with a reduced set of data before running the full task. You can use the `sample()` method to extract just a sample of the data or use 

These two RDD's are called "rawCitations" and "rawPatents" because you probably want to process them futher (e.g. convert them to integer types, etc). 

The `textFile` function returns data in strings. This should work fine for this lab.

Other methods you use might return data in type `Byte`. If you haven't used Python `Byte` types before, google it. You can convert a value of `x` type byte into e.g. a UTF8 string using `x.decode('uft-8')`. Alternatively, you can use the `open` method of the gzip library to read in all the lines as UTF-8 strings like this:
```
import gzip
with gzip.open('cite75_99.txt.gz', 'rt',encoding='utf-8') as f:
    rddCitations = sc.parallelize( f.readlines() )
```
This is less efficient than using `textFile` because `textFile` would use the underlying HDFS or other file system to read the file across all the worker nodes while the using `gzip.open()...readlines()` will read all the data in the frontend and then distribute it to all the worker nodes.

In [3]:
rddCitations = sc.textFile("cite75_99.txt.gz")
rddPatents = sc.textFile("apat63_99.txt.gz")

The data looks like the following.

In [156]:
rddCitations.take(5)

['"CITING","CITED"',
 '3858241,956203',
 '3858241,1324234',
 '3858241,3398406',
 '3858241,3557384']

In [157]:
rddPatents.take(5)

['"PATENT","GYEAR","GDATE","APPYEAR","COUNTRY","POSTATE","ASSIGNEE","ASSCODE","CLAIMS","NCLASS","CAT","SUBCAT","CMADE","CRECEIVE","RATIOCIT","GENERAL","ORIGINAL","FWDAPLAG","BCKGTLAG","SELFCTUB","SELFCTLB","SECDUPBD","SECDLWBD"',
 '3070801,1963,1096,,"BE","",,1,,269,6,69,,1,,0,,,,,,,',
 '3070802,1963,1096,,"US","TX",,1,,2,6,63,,0,,,,,,,,,',
 '3070803,1963,1096,,"US","IL",,1,,2,6,63,,9,,0.3704,,,,,,,',
 '3070804,1963,1096,,"US","OH",,1,,2,6,63,,3,,0.6667,,,,,,,']

In other words, they are a single string with multiple CSV's. You will need to convert these to (K,V) pairs, probably convert the keys to `int` and so on. You'll need to `filter` out the header string as well since there's no easy way to extract all the lines except the first.

First extract the first row with header

In [4]:
patentHeader = rddPatents.first()


citationHeader = rddCitations.first()


In [5]:
rddPatentsNoHeader = rddPatents.filter(lambda x: x != patentHeader).cache()
rddCitationsNoHeader = rddCitations.filter(lambda x: x != citationHeader).cache()

In [6]:
#rddPatentsNoHeader=rddPatentsNoHeader.sample(False, 0.1)
#rddCitationsNoHeader=rddCitationsNoHeader.sample(False, 0.1)

Get column headers in list format

In [7]:
patentHeader = patentHeader.split(",")
patentHeader = [x.strip("\"") for x in patentHeader]

citationHeader = citationHeader.split(",")
citationHeader = [x.strip("\"") for x in citationHeader]

Get patent info as (patent_id, {header: value})

In [8]:
rddPatentsSplit = rddPatentsNoHeader.map(lambda x: (int(x.split(",")[0].strip()), {patentHeader[i+1]: int(val.strip()) if val.strip().isdigit() else val.strip("\"") for i, val in enumerate(x.split(",")[1:])}))

In [9]:
rddPatentsSplit.take(5)

[(3070801,
  {'GYEAR': 1963,
   'GDATE': 1096,
   'APPYEAR': '',
   'COUNTRY': 'BE',
   'POSTATE': '',
   'ASSIGNEE': '',
   'ASSCODE': 1,
   'CLAIMS': '',
   'NCLASS': 269,
   'CAT': 6,
   'SUBCAT': 69,
   'CMADE': '',
   'CRECEIVE': 1,
   'RATIOCIT': '',
   'GENERAL': 0,
   'ORIGINAL': '',
   'FWDAPLAG': '',
   'BCKGTLAG': '',
   'SELFCTUB': '',
   'SELFCTLB': '',
   'SECDUPBD': '',
   'SECDLWBD': ''}),
 (3070802,
  {'GYEAR': 1963,
   'GDATE': 1096,
   'APPYEAR': '',
   'COUNTRY': 'US',
   'POSTATE': 'TX',
   'ASSIGNEE': '',
   'ASSCODE': 1,
   'CLAIMS': '',
   'NCLASS': 2,
   'CAT': 6,
   'SUBCAT': 63,
   'CMADE': '',
   'CRECEIVE': 0,
   'RATIOCIT': '',
   'GENERAL': '',
   'ORIGINAL': '',
   'FWDAPLAG': '',
   'BCKGTLAG': '',
   'SELFCTUB': '',
   'SELFCTLB': '',
   'SECDUPBD': '',
   'SECDLWBD': ''}),
 (3070803,
  {'GYEAR': 1963,
   'GDATE': 1096,
   'APPYEAR': '',
   'COUNTRY': 'US',
   'POSTATE': 'IL',
   'ASSIGNEE': '',
   'ASSCODE': 1,
   'CLAIMS': '',
   'NCLASS': 2,
   'CAT

Get only state patent

In [10]:
rddPatentState = rddPatentsSplit.map(lambda x: (x[0], x[1]['POSTATE'])) # (PATENT, POSTATE)

Filter out null values

In [11]:
rddPatentState = rddPatentState.filter(lambda x: x[1] != None)

In [12]:
rddPatentState = rddPatentState.filter(lambda x: x[1] != "").cache()

In [13]:
rddPatentState.take(5)

[(3070802, 'TX'),
 (3070803, 'IL'),
 (3070804, 'OH'),
 (3070805, 'CA'),
 (3070806, 'PA')]

Get citing info

In [14]:
rddCitationsSplit = rddCitationsNoHeader.map(lambda x: {citationHeader[i]: int(val.strip()) if val.strip().isdigit() else val.strip("\"") for i, val in enumerate(x.split(","))})
rddCitationsDict = rddCitationsSplit.map(lambda x: {key: (None if value == "" else value) for key, value in x.items()})
rddCitationsDict.take(5)



[{'CITING': 3858241, 'CITED': 956203},
 {'CITING': 3858241, 'CITED': 1324234},
 {'CITING': 3858241, 'CITED': 3398406},
 {'CITING': 3858241, 'CITED': 3557384},
 {'CITING': 3858241, 'CITED': 3634889}]

Separate in two rdds for easier work

In [18]:
rddCiting = rddCitationsSplit.map(lambda x: (x['CITING'], x)).cache() 
rddCited = rddCitationsSplit.map(lambda x: (x['CITED'], x)).cache()




Join patent and citing rdds with patent id key

In [19]:
rddCitationsState = rddCiting.join(rddPatentState).cache()

In [20]:
rddCitationsState.take(5)

[(4598722, ({'CITING': 4598722, 'CITED': 2742046}, 'PA')),
 (4598722, ({'CITING': 4598722, 'CITED': 2775244}, 'PA')),
 (4598722, ({'CITING': 4598722, 'CITED': 4202360}, 'PA')),
 (4598722, ({'CITING': 4598722, 'CITED': 4267851}, 'PA')),
 (4598722, ({'CITING': 4598722, 'CITED': 4453554}, 'PA'))]

Reformat

In [21]:
rddCitingWithState = rddCitationsState.map(lambda x: (
    x[0],  # CITING
    (x[1][0]['CITED'], x[1][1])  # (CITED, CITING_STATE)
)).cache()

In [22]:
rddCitingWithState.take(5)

[(4598722, (2742046, 'PA')),
 (4598722, (2775244, 'PA')),
 (4598722, (4202360, 'PA')),
 (4598722, (4267851, 'PA')),
 (4598722, (4453554, 'PA'))]

Now for cited

In [23]:
rddCitedState = rddCited.join(rddPatentState).cache()

Reformat

In [24]:
rddCitedStateKV = rddCitedState.map(lambda x: (
    x[1][0]['CITED'],  # CITED from citation dict
    x[1][1]  # CITED_STATE
)).cache()

Join citing and cited

In [25]:
rddWithBothStates = rddCitingWithState.map(lambda x: (x[1][0], (x[0], x[1][1]))).join(rddCitedStateKV).cache()
# (CITING, ((CITED, CITING_STATE), CITED_STATE))

In [26]:
rddWithBothStates.take(5)

[(3677436, ((3878980, 'CA'), 'MN')),
 (3677436, ((3878980, 'CA'), 'MN')),
 (3677436, ((3878980, 'CA'), 'MN')),
 (3677436, ((3878980, 'CA'), 'MN')),
 (3677436, ((3878980, 'CA'), 'MN'))]

Apply filter for same state and sum total values

In [27]:
rddSameStateCount = rddWithBothStates.filter(
    lambda x: x[1][0][1] == x[1][1] and x[1][1] is not None  # CITING_STATE == CITED_STATE and not None
).map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)

In [28]:
rddSameStateCount.take(5)

[(3675908, 14), (4029756, 84), (3439656, 20), (4337116, 44), (4460924, 64)]

In [29]:
rddFinal = rddPatentsSplit.join(rddSameStateCount).cache()

Sort by count and join with original patent rdd

In [30]:
rddFinal rddFinal.sortBy(lambda x: x[1][1], ascending=False).map(lambda x: (x[0], x[1][0]|{"CITATION_COUNT": x[1][1]})).first()

(4683195,
 {'GYEAR': 1987,
  'GDATE': 10070,
  'APPYEAR': 1986,
  'COUNTRY': 'US',
  'POSTATE': 'CA',
  'ASSIGNEE': 97235,
  'ASSCODE': 2,
  'CLAIMS': 26,
  'NCLASS': 435,
  'CAT': 3,
  'SUBCAT': 33,
  'CMADE': 1,
  'CRECEIVE': 631,
  'RATIOCIT': 1,
  'GENERAL': '0.3464',
  'ORIGINAL': 0,
  'FWDAPLAG': '7.8494',
  'BCKGTLAG': 4,
  'SELFCTUB': 0,
  'SELFCTLB': 0,
  'SECDUPBD': '0.0183',
  'SECDLWBD': '0.0174',
  'CITATION_COUNT': 134403})